## É um carro?

Vamos criar um modelo que reconece se uma imagem contém ou não um carro

## Step 1: Fazer o download de imagens de carros e rodovias

Vamos começar instalando a ferramenta que irá pesquisar as imagens

In [ ]:
!pip install -U duckduckgo_search

In [ ]:
from duckduckgo_search import ddg_images
from fastcore.all import *

def search_images(term, max_images=200): return L(ddg_images(term, max_results=max_images)).itemgot('image')

Pesquisar uma imagem de carro:

In [ ]:
urls = search_images('car photos', max_images=1)
urls[0]

Fazer o download e exibir a imagem

In [ ]:
from fastdownload import download_url
dest = 'car.jpg'
download_url(urls[0], dest, show_progress=False)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)

Agora o mesmo com as imagens de rodovias:

In [ ]:
download_url(search_images('highway photos', max_images=1)[0], 'highway.jpg', show_progress=False)
Image.open('highway.jpg').to_thumb(256,256)

Agora vamos fazer o download de 200 exemplares de imagens de carros e estradas:

In [ ]:
searches = 'highway','car'
path = Path('is_that_a_car')
from time import sleep

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o} photo'))
    sleep(10)  # Pause between searches to avoid over-loading server
    download_images(dest, urls=search_images(f'{o} sun photo'))
    sleep(10)
    download_images(dest, urls=search_images(f'{o} shade photo'))
    sleep(10)
    resize_images(path/o, max_size=400, dest=path/o)

## Step 2: Treinando o modelo

Algumas imagens podem não ser baixadas da maneira correta, aqui vamos removê-las:

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

To train a model, we'll need `DataLoaders`, which is an object that contains a *training set* (the images used to create a model) and a *validation set* (the images used to check the accuracy of a model -- not used during training). In `fastai` we can create that easily using a `DataBlock`, and view sample images from it:

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path)

dls.show_batch(max_n=6)

O que cada parâmetro `DataBlock` significa:

    blocks=(ImageBlock, CategoryBlock),

As entradas do modelo são imagens,e as saídas são categorias(carro ou rodovia).

    get_items=get_image_files, 

Para encontrar todas as entradas do modelo, executar a função: `get_image_files`, que retorna uma lista de todas as imagens na pasta.

    splitter=RandomSplitter(valid_pct=0.2, seed=42),

Dividir os dados em treinamento e validação aleatoriamente, usando 20% dos dados para a validação.

    get_y=parent_label,

O rótulo (`y` values) é o nome do `parent` de cada arquivo.

    item_tfms=[Resize(192, method='squish')]

Antes do treinamento, as imagens são redimensionadas para 192x192 pixels, utilizando ""squishing"

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

## Step 3: Usando o modelo

Vamos ver o que o modelo diz sobre imagem de carro baixada no começo:

In [ ]:
is_that_a_car,_,probs = learn.predict(PILImage.create('car.jpg'))

print(f"This is a: {is_that_a_car}.")
print(f"Probability it's a car: {probs[0]:.4f}")

Agora vamos ver o que o modelo pensa sobre a imagem da rodovia baixada no começo:

In [ ]:
is_that_a_car,_,probs = learn.predict(PILImage.create('highway.jpg'))

print(f"This is a: {is_that_a_car}.")
print(f"Probability it's a car: {probs[0]:.4f}")